In [2]:
import pandas as pd
import requests
import json

In [10]:
METADATA_API = "https://maayanlab.cloud/sigcom-lincs/metadata-api/"
DATA_API = "https://maayanlab.cloud/sigcom-lincs/data-api/api/v1/"

input_gene_set = {
#     # CMS3 colotype
#     "up_genes": ["CAPN9", "B3GNT6", "FCGBP", "RASD1", "ASRGL1", "SPINK4"],
#     "down_genes": ["TIMP3", "VAV2", "RBMS1", "FBN1", "TBX10"]
    
#     # CMS3 colotype + GALNT12 + MLH1 
#     "up_genes": ["CAPN9", "B3GNT6", "FCGBP", "RASD1", "ASRGL1", "SPINK4", 'GALNT12'],
#     "down_genes": ["TIMP3", "VAV2", "RBMS1", "FBN1", "TBX10", 'MLH1']
    
    # CMS3 colotype + GALNT12
    "up_genes": ["CAPN9", "B3GNT6", "FCGBP", "RASD1", "ASRGL1", "SPINK4", 'GALNT12'],
    "down_genes": ["TIMP3", "VAV2", "RBMS1", "FBN1", "TBX10",]
}

all_genes = input_gene_set["up_genes"] + input_gene_set["down_genes"]

payload = {
    "filter": {
        "where": {
            "meta.symbol": {
                "inq": all_genes
            }
        },
        "fields": ["id", "meta.symbol"]
    }
}
res = requests.post(METADATA_API + "entities/find", json=payload)
entities = res.json()

for_enrichment = {
    "up_entities": [],
    "down_entities": []
}

for e in entities:
    symbol = e["meta"]["symbol"]
    if symbol in input_gene_set["up_genes"]:
        for_enrichment["up_entities"].append(e["id"])
    elif symbol in input_gene_set["down_genes"]:
        for_enrichment["down_entities"].append(e["id"])

print(json.dumps(for_enrichment, indent=2))

{
  "up_entities": [
    "d8d54ae9-ffb7-4e8b-883f-f26ccb810927",
    "91223623-7063-4424-b7f6-15b7b89bcf89",
    "0f42c603-c167-48bf-b847-e9e82bbf5f75",
    "a5a07ec6-3dfc-464d-af32-5d74fb9c605f",
    "ef2cfbe2-b580-52d0-a4b6-0c10671a5b15",
    "cbec4707-40a8-4e6d-9d3c-bcdc1f5e2b97"
  ],
  "down_entities": [
    "624778e8-5e87-43c0-acda-7e8e9fbf73d8",
    "2ce7dc09-d2d8-44a3-950d-98b527ef182f",
    "ca89292a-524f-40fd-bff1-1f544a7f411c",
    "9322b2bb-0917-44ec-b5d7-965edfa3ceb8",
    "86e2d176-19ec-40c3-ae0f-77e014fbb5ae"
  ]
}


In [11]:
payload = {
    "meta": {
        "$validator": "/dcic/signature-commons-schema/v6/meta/user_input/user_input.json",
        **for_enrichment
    },
    "type": "signature"
}
res = requests.post(METADATA_API + "user_input", json=payload)
persistent_id = res.json()["id"]
print("Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/%s"%persistent_id)

Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/75da5bfc-6f55-56a4-9d32-322111a6fd9c
